In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras import losses, optimizers

from awkde import GaussianKDE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Data
data_dir = '../data'

X_train = np.load(data_dir + '/unweighted_events/X_train_scoreregression.npy')
scores_train = np.load(data_dir + '/unweighted_events/scores_train_scoreregression.npy')

X_calibration = np.load(data_dir + '/unweighted_events/X_calibration.npy')
weights_calibration = np.load(
    data_dir + '/unweighted_events/weights_calibration.npy')
    
# Scale data
scaler = StandardScaler()
scaler.fit(np.array(X_train, dtype=np.float64))
X_train_transformed = scaler.transform(X_train)
X_calibration_transformed = scaler.transform(X_calibration)

In [4]:
theta1 = 708
theta = 0

# Score regression

In [5]:
def make_regressor(n_hidden_layers=3,
                    hidden_layer_size=100,
                    activation='tanh',
                    dropout_prob=0.0):
    # Inputs
    input_layer = Input(shape=(42,))

    # Network
    hidden_layer = Dense(hidden_layer_size, activation=activation)(input_layer)
    if n_hidden_layers > 1:
        hidden_layer_ = hidden_layers(n_hidden_layers - 1,
                                      hidden_layer_size=hidden_layer_size,
                                      activation=activation,
                                      dropout_prob=dropout_prob)
        hidden_layer = hidden_layer_(hidden_layer)
    score_layer = Dense(2, activation='linear')(hidden_layer)

    # Combine outputs
    model = Model(inputs=[input_layer], outputs=[score_layer])

    # Compile model
    model.compile(loss='mean_squared_error',
                  optimizer=optimizers.Adam(clipnorm=1.))

    return model

In [6]:
# Train score regression
regr = KerasRegressor(lambda: make_regressor(n_hidden_layers=1),
                      epochs=1, validation_split=0.142857,
                      verbose=2)

regr.fit(X_train_transformed[::10], scores_train[::10])

Train on 857123 samples, validate on 142854 samples
Epoch 1/1
 - 23s - loss: 0.1079 - val_loss: 0.0637


In [7]:
that_calibration = regr.predict(X_calibration_transformed)

# Calibration

In [8]:
pdf_nom = GaussianKDE()
print(that_calibration[::].shape)
print(weights_calibration[theta,::].shape)
pdf_nom.fit(that_calibration[::], weights=weights_calibration[theta,::])

pdf_den = GaussianKDE()
pdf_den.fit(that_calibration[::], weights=weights_calibration[theta1,::])

(19994, 2)
(19994,)


(array([0.48151085, 0.30060332]), array([[ 5.12649046, -0.11479686],
        [-0.11479686,  3.18356221]]))

# Plot pdfs and ratio

In [ ]:
xi = np.linspace(-2.0, 2.0, 100)
yi = np.linspace(-2.0, 2.0, 100)
xx, yy = np.meshgrid(xi, yi)
scores_eval = np.asarray((xx.flatten(), yy.flatten())).T

In [ ]:
p_nom_eval = pdf_nom.predict(scores_eval)
p_den_eval = pdf_den.predict(scores_eval)
ratio_eval = p_nom_eval / p_den_eval

In [ ]:
plt.figure(figsize=(15.,4.))



plt.subplot(1,3,1)
cs = plt.contourf(xi, yi, (p_nom_eval).reshape((100, 100)), 100, cmap="viridis")
cbar = plt.colorbar()
plt.xlabel('$t_0$')
plt.ylabel('$t_1$')
cbar.set_label(r'$\log p(t|\theta_0)$')



plt.subplot(1,3,2)
cs = plt.contourf(xi, yi, (p_den_eval).reshape((100, 100)), 100, cmap="viridis")
cbar = plt.colorbar()
plt.xlabel('$t_0$')
plt.ylabel('$t_1$')
cbar.set_label(r'$\log p(t|\theta_1)$')



plt.subplot(1,3,3)
cs = plt.contourf(xi, yi, (ratio_eval).reshape((100, 100)), 100, cmap="viridis")
cbar = plt.colorbar()
plt.xlabel('$t_0$')
plt.ylabel('$t_1$')
cbar.set_label(r'$\log r(t; \theta_0, \theta_1)$')


plt.tight_layout()
plt.show()